# Load MNIST data

https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py

In [96]:
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, SeparableConv2D, GlobalAveragePooling2D, Activation, Multiply
from keras.optimizers import RMSprop
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 3

In [29]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


# Normal CNN

In [84]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_43 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_26 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_77 (Dense)             (None, 10)                92170     
Total params: 110,986
Trainable params: 110,986
Non-trainable params: 0
_________________________________________________________________


In [85]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 182s 3ms/step - loss: 0.1890 - acc: 0.9431 - val_loss: 0.0634 - val_acc: 0.9783
Epoch 2/3
60000/60000 [==============================] - 204s 3ms/step - loss: 0.0636 - acc: 0.9811 - val_loss: 0.0470 - val_acc: 0.9848
Epoch 3/3
60000/60000 [==============================] - 218s 4ms/step - loss: 0.0479 - acc: 0.9861 - val_loss: 0.0457 - val_acc: 0.9839
Test loss: 0.045710997666278855
Test accuracy: 0.9839


# Depthwise Separable CNN

https://keras.io/ja/layers/convolutional/

In [83]:
# Mobile net
model = Sequential()
model.add(SeparableConv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(SeparableConv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_38 (Separab (None, 26, 26, 32)        73        
_________________________________________________________________
separable_conv2d_39 (Separab (None, 24, 24, 64)        2400      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_42 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_76 (Dense)             (None, 10)                92170     
Total params: 94,643
Trainable params: 94,643
Non-trainable params: 0
_________________________________________________________________


In [59]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 106s 2ms/step - loss: 0.3548 - acc: 0.8994 - val_loss: 0.1664 - val_acc: 0.9492
Epoch 2/3
60000/60000 [==============================] - 107s 2ms/step - loss: 0.1569 - acc: 0.9531 - val_loss: 0.1262 - val_acc: 0.9621
Epoch 3/3
60000/60000 [==============================] - 106s 2ms/step - loss: 0.1129 - acc: 0.9659 - val_loss: 0.0968 - val_acc: 0.9699
Test loss: 0.09679398665204644
Test accuracy: 0.9699


# Depthwise Separable + SE CNN

https://keras.io/ja/layers/pooling/

In [104]:
# Mobile + SE net
reduction_rate = 16

inputs = Input(shape=(28,28,1))
x = SeparableConv2D(32, kernel_size=(3, 3), activation=None)(inputs)
se = GlobalAveragePooling2D()(x)
se = Dense(32//reduction_rate, activation='relu')(se)
se = Dense(32, activation='sigmoid')(se)
x = Multiply()([x, se])
x = Activation('relu')(x)
x = SeparableConv2D(64, kernel_size=(3, 3), activation=None)(x)
se = GlobalAveragePooling2D()(x)
se = Dense(64//reduction_rate, activation='relu')(se)
se = Dense(64, activation='sigmoid')(se)
x = Multiply()([x, se])
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(10, activation='softmax')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
separable_conv2d_58 (SeparableC (None, 26, 26, 32)   73          input_26[0][0]                   
__________________________________________________________________________________________________
global_average_pooling2d_31 (Gl (None, 32)           0           separable_conv2d_58[0][0]        
__________________________________________________________________________________________________
dense_130 (Dense)               (None, 2)            66          global_average_pooling2d_31[0][0]
__________________________________________________________________________________________________
dense_131 

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 202s 3ms/step - loss: 1.7807 - acc: 0.5783 - val_loss: 1.4940 - val_acc: 0.6412
Epoch 2/3
14848/60000 [======>.......................] - ETA: 2:42 - loss: 1.4226 - acc: 0.6555